# Custom Chatbot Notebook

An OpenAI client is initialised by using environment variables and a tokenizer is set up for a specific model (`gpt-4o-mini-2024-07-18`). Also,the necessary libraries and custom utility functions are imported.

In [1]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken

# Custom Functions
from fncs.utilities import (
    create_openai_client,
    response_generator,
    prompt_builder,
    calculate_total_cost
    )
from fncs.retrieval import (
    get_embedding,
    search_text,
    control_chunk_context
    )

from fncs.prompt_templates import (
    user_prompt,
    user_prompt_without_context
)

# Load environment vars:
load_dotenv()
base_url_voc = os.getenv("OPENAI_BASE_VOC")
api_key_voc = os.getenv("OPENAI_API_VOC")
# Deployment model names
chat_name = 'gpt-4o' # 'gpt-4o-mini-2024-07-18' # 'gpt-4o-mini'
emb_name = 'text-embedding-3-large'
# Initialising OpenAI client
openai_client = create_openai_client(api_key= api_key_voc, base_url= base_url_voc)
tokenizer = tiktoken.encoding_for_model(chat_name)

### Loading dataset

In [2]:
proj_dir = Path(os.getcwd())
df = pd.read_csv(proj_dir / "data" / "2023_fashion_trends_embeddings.csv")
df.head(3)

,text,embeddings
0,\nFashion trends according to refinery29\n\nSo...,"[-0.06195216625928879, -0.007596897892653942, ..."
1,\nFashion trends according to refinery29\n\nSo...,"[-0.0732918530702591, -0.014976361766457558, -..."
2,\nFashion trends according to refinery29\n\nSo...,"[-0.05401710420846939, -0.003982114605605602, ..."


The embeddings are stored as text/string in the DataFrame and need to be converted to lists/arrays

In [3]:
import ast
# Converting the string representations of embeddings to actual lists
df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

Checking transformation

In [4]:
type(df[['embeddings']].iloc[0].values[0])

list

### Calculating Cosine Distances based on query


Below I create a query string about fashion trends in 2023. Then, by using the `get_embedding` function, the embeddings of the query are generated, by passing the query, OpenAI client, and embedding model as inputs.

In [5]:
query = "What is the most popular fashion trend about pants in 2023?"
query_emb = get_embedding(text=query, client = openai_client, model=emb_name)

The DataFrame `df` is sorted based on the cosine distance between the query embedding (`query_emb`) and the embeddings in the DataFrame using the `search_text` function, and stores the result in `df_sorted`.

In [6]:
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

In [7]:
df_sorted.head()

,text,embeddings,distance
1,\nFashion trends according to refinery29\n\nSo...,"[-0.0732918530702591, -0.014976361766457558, -...",0.288696
58,\nFashion trends according to whowhatwear\n\nS...,"[-0.05073591694235802, -0.018371405079960823, ...",0.319315
3,\nFashion trends according to refinery29\n\nSo...,"[-0.052798643708229065, -0.02721790410578251, ...",0.326079
44,\nFashion trends according to whowhatwear\n\nS...,"[-0.05533209815621376, -0.032554663717746735, ...",0.329414
45,\nFashion trends according to whowhatwear\n\nS...,"[-0.024114221334457397, -0.018797021359205246,...",0.349743


### Prompt Template



Creating the system prompt to be used in the chatbot

In [8]:
system_prompt = "You are an expert fashion trend analyser. Based only on the provided information you must analyse and summarise the trends and provide an accurate answer."

print(f"System Prompt Tokens: {len(tokenizer.encode(system_prompt))}")

System Prompt Tokens: 28


Calling the user prompt templates functions that will

In [9]:
print(f"User Prompt (with context) Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt()))}")

User Prompt (with context) Tokens BEFORE context insertion: 130


In [10]:
# to be used in performance demonstration later
print(f"User Prompt (without context) Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt_without_context()))}")

User Prompt (without context) Tokens BEFORE context insertion: 94


#### Apply token controller function ( fnc: control_chunk_context )

The variable `max_token_count` to 1000, serves as a limit for the total number of tokens allowed in a prompt.

In [11]:
#parameter that control the prompt tokens:
max_token_count = 1000

The code below calculates the current token count of the prompts (system and user) and generates a context by selecting data from the sorted DataFrame (`df_sorted`) based on a maximum allowed token limit (`max_token_count`) using the `control_chunk_context` function.

In [12]:
current_token_count = len(tokenizer.encode(user_prompt())) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(
    df_sorted,
    current_token_count,
    max_token_count,
    tokenizer = tokenizer
)

 Below, the final `user_prompt` is created by inserting the generated `context` into the prompt template and by formatting it with the query and context.

In [13]:
# prompt template params
context_inprompt = "\n----\n".join(context)
user_prompt_0 = user_prompt().format(query, context_inprompt)

print(user_prompt_0)


    ***Question: What is the most popular fashion trend about pants in 2023?
    
    ***Context:
    <--Start of Context-->
    
Fashion trends according to refinery29

Source Title: 7 Fashion Trends That Will Take Over 2023 — Shop Them Now

2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, this trend is translated into tailored silhouettes, interesting pocket placements, elevated fabrics like silk and organza, and colors that go beyond khaki and olive.

Source URL: https://www.refinery29.com/en-us/fashion-trends-2023

----

Fashion trends according to whowhatwear

Source Title: Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See

Every buyer I have spoken to has been most excited by the many pairs of perfectly cut trousers in the spring/summer 2023 collections, which actually should hardly come as a surprise. It's b

In [14]:
print(f"User Prompt Tokens AFTER context insertion: {len(tokenizer.encode(user_prompt_0))}")

User Prompt Tokens AFTER context insertion: 763


## Custom Query Completion

**Finally, the code below generates a final prompt using the `prompt_builder` function by combining the system and user prompts. It then sends the prompt to the OpenAI model (`chat_model`) using the `response_generator` function with specified additional options (e.g., `temperature=0.1`) to generate an AI response. It also calculates the total cost in EUR based on the API usage (`response_full.usage`) for the specific deployment (`gpt-4o`).**

In [15]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_0)
additional_options = {"temperature": 0.1,}
response, response_full = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur} eur')

Query Completion Total Cost is: 0.00330951342 eur


In [16]:
print(response)

1. Answer:
   The most popular fashion trend about pants in 2023 is the return of cargo pants with a modern twist, featuring tailored silhouettes, interesting pocket placements, and elevated fabrics. Additionally, perfectly cut trousers, including wide-leg and slouchy-fit styles, are also trending, with designers becoming more playful with their designs.

2. Sources:
   • https://www.refinery29.com/en-us/fashion-trends-2023
   • https://www.whowhatwear.com/spring-summer-2023-fashion-trends/


In [17]:
print('Total Tokens: ', response_full.usage.total_tokens)
print('Total Completion Tokens: ', response_full.usage.completion_tokens)
print('Total Prompt Tokens: ', response_full.usage.prompt_tokens)

Total Tokens:  915
Total Completion Tokens:  113
Total Prompt Tokens:  802


## Demonstrating Performance

Below I demonstrate through some examples how using a custom prompt significantly enhances the performance and accuracy of responses from the OpenAI model compared to basic (generic) prompts. I showcase two different example queries about 2023 fashion trends, providing the responses produced using the custom context-based prompt and a basic prompt (without context).

In other words, the comparison made below, clearly illustrates that custom prompts, enriched with relevant contextual information, greatly enhance the accuracy, specificity, and usefulness of model-generated responses. Basic prompts, without contextual enrichment, yield generic and less informative answers.

### Question 1
**Question**: According to Vogue, what is a new trend presented by Prada on New York Fashion Week?

In [18]:
query_1 = "According to Vogue, what is a new trend presented by Prada on New York Fashion Week?"
max_token_count = 1000

1.1 Using custom prompt with context:

In [19]:
query_emb = get_embedding(text=query_1, client = openai_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt())) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_1 = user_prompt().format(query_1, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_1)
additional_options = {"temperature": 0.1,}

response_1_1, response_full_1_1 = \
    response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

cost_eur_1_1 = \
    calculate_total_cost(response_usage= response_full.usage, deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_1_1} eur')


Query Completion Total Cost is: 0.00330951342 eur


In [20]:
print(response_1_1)

1. Answer:
   Prada presented a trend of "Perfectly Imperfect" during New York Fashion Week, featuring a satin midi skirt that evokes a sense of "unfinishedness" with an irregularly dyed print and a slit designed to look like the skirt is torn.

2. Sources:
   • https://www.vogue.com/article/spring-2023-trends-editors-picks


1.2 Using basic prompt with no context:

In [21]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_without_context().format(query_1) )
additional_options = {"temperature": 0.1,}

response_1_2, response_full_1_2 = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur_1_2 = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_1_2} eur')

Query Completion Total Cost is: 0.00330951342 eur


In [22]:
print(response_1_2)

1. Answer:
   Cannot be determined as I do not have enough the knowledge to answer this question.

2. Sources:
   • [Not available]
   • [Not available]


### Question 2
**Question**: According to glamour magazine and whowhatwear.com what are the denim fashion trends for the year 2023?

In [23]:
query_2 = "According to glamour magazine and whowhatwear.com what are the denim fashion trends for the year 2023?"
max_token_count = 1000

2.1 Using custom prompt with context:

In [24]:
query_emb = get_embedding(text=query_2, client = openai_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt())) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_2 = user_prompt().format(query_2, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_2)
additional_options = {"temperature": 0,}

response_2_1, response_full_2_1 = \
    response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

cost_eur_2_1 = \
    calculate_total_cost(response_usage= response_full.usage, deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_2_1} eur')

Query Completion Total Cost is: 0.00330951342 eur


In [25]:
print(response_2_1)

1. Answer:
   According to Glamour magazine, the denim fashion trend for 2023 is baggy denim. Denim remains as baggy as it has been, if not even looser, with a great light-wash pair of jeans offering endless styling potential. 

   According to WhoWhatWear, the trend is towards more relaxed silhouettes in denim, with styles so relaxed they might as well be joggers. The trend is moving away from skinny jeans to wide-leg trousers.

2. Sources:
   • https://www.glamour.com/story/spring-fashion-trends
   • https://www.whowhatwear.com/spring-summer-2023-fashion-trends/


2.2 Using basic prompt with no context:

In [26]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_without_context().format(query_2) )
additional_options = {"temperature": 0,}

response_2_2, response_full_2_2 = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)

cost_eur_2_2 = calculate_total_cost(response_usage= response_full.usage,deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_2_2} eur')

Query Completion Total Cost is: 0.00330951342 eur


In [27]:
print(response_2_2)

1. Answer:
   The denim fashion trends for 2023, as reported by Glamour Magazine and Who What Wear, include a focus on baggy and relaxed fits, low-rise jeans making a comeback, and the popularity of cargo-style denim. Additionally, there is an emphasis on vintage-inspired washes and patchwork designs, as well as the continued presence of straight-leg and wide-leg silhouettes.

2. Sources:
   • Cannot provide specific URLs as I do not have access to external content.


In [28]:
print('Notebook Finished')

Notebook Finished
